# การคำนวณค่าใช้จ่าย (Cost Function) สำหรับ Logistic Regression: การทดลองเสริม

## เป้าหมาย:
ในการทดลองนี้ คุณจะ:
- ศึกษาการใช้งานและประยุกต์ใช้ฟังก์ชันค่าใช้จ่ายสำหรับ Logistic Regression








In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
try:
  %matplotlib widget
  print("widget is already installed")
except:
  print("widget is not been installed, install now..")
  !pip install ipympl

In [ ]:
# prompt: import requsts and download from this github link : https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle

import requests
from pathlib import Path

url = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/deeplearning.mplstyle'
url2 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/plt_logistic_loss.py'
url3 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Machine_Learning_Course/main/work/lab_utils_common_c3.py'

response = requests.get(url)
with open('deeplearning.mplstyle', 'wb') as f:
  f.write(response.content)

response = requests.get(url2)
with open('plt_logistic_loss.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url3)
with open('lab_utils_common.py', 'wb') as f:
  f.write(response.content)

In [ ]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from lab_utils_common import  plot_data, sigmoid, dlc
plt.style.use('./deeplearning.mplstyle')

## ชุดข้อมูล (Dataset)
เราจะเริ่มต้นด้วยชุดข้อมูลเดียวกันกับที่ใช้ในการทดลองเกี่ยวกับขอบเขตการตัดสินใจ (decision boundary)


In [ ]:
X_train = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])  #(m,n)
y_train = np.array([0, 0, 0, 1, 1, 1])                                           #(m,)

เราจะใช้ฟังก์ชันช่วยเพื่อพล็อตข้อมูลนี้ จุดข้อมูลที่มีป้ายกำกับ  $y=1$ จะแสดงเป็นกากบาทสีแดง ขณะที่จุดข้อมูลที่มีป้ายกำกับ $y=0$ จะแสดงเป็นวงกลมสีน้ำเงิน

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,4))
plot_data(X_train, y_train, ax)

# Set both axes to be from 0-4
ax.axis([0, 4, 0, 3.5])
ax.set_ylabel('$x_1$', fontsize=12)
ax.set_xlabel('$x_0$', fontsize=12)
plt.show()

## ฟังก์ชันต้นทุน (Cost function)

ใน lab ก่อนหน้านี้ คุณได้พัฒนา ฟังก์ชันการสูญเสียแบบโลจิสติก *(logistic loss function)*การทบทวน: ฟังก์ชันการสูญเสีย (loss) ถูกกำหนดให้ใช้กับตัวอย่างหนึ่ง
ในที่นี้ คุณจะรวมฟังก์ชันการสูญเสียทั้งหมดเพื่อสร้าง *ฟังก์ชันต้นทุน (cost)* ซึ่งรวมถึงตัวอย่างทั้งหมด

ในการถดถอยโลจิสติก ฟังก์ชันต้นทุน (cost function) จะอยู่ในรูปแบบดังนี้

$$ J(\mathbf{w},b) = \frac{1}{m} \sum_{i=0}^{m-1} \left[ loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) \right] \tag{1}$$

โดยที่
* $loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)})$ is the cost for a single data point, which is:

    $$loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = -y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \tag{2}$$
    
*  เมื่อ m คือจำนวนตัวอย่างการฝึกในชุดข้อมูล และ:

$$
\begin{align}
  f_{\mathbf{w},b}(\mathbf{x^{(i)}}) &= g(z^{(i)})\tag{3} \\
  z^{(i)} &= \mathbf{w} \cdot \mathbf{x}^{(i)}+ b\tag{4} \\
  g(z^{(i)}) &= \frac{1}{1+e^{-z^{(i)}}}\tag{5} 
\end{align}
$$
 

<a name='ex-02'></a>
#### คำอธิบายโค้ด


อัลกอริทึม  `compute_cost_logistic` ทำการวนลูปผ่านตัวอย่างทั้งหมด คำนวณค่าสูญเสีย (loss) สำหรับแต่ละตัวอย่าง และสะสมผลรวมทั้งหมด

โปรดสังเกตว่าตัวแปร X และ  y ไม่ใช่ค่าสเกลาร์ แต่เป็นเมทริกซ์ที่มีขนาด ($m, n$) และ  ($𝑚$,)  ตามลำดับ โดย $𝑛$ คือจำนวนฟีเจอร์ และ $𝑚$ คือจำนวนตัวอย่างการฝึก


In [ ]:
def compute_cost_logistic(X, y, w, b):
    """
    Computes cost

    Args:
      X (ndarray (m,n)): Data, m examples with n features
      y (ndarray (m,)) : target values
      w (ndarray (n,)) : model parameters  
      b (scalar)       : model parameter
      
    Returns:
      cost (scalar): cost
    """

    m = X.shape[0]
    cost = 0.0
    for i in range(m):
        z_i = np.dot(X[i],w) + b
        f_wb_i = sigmoid(z_i)
        cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)
             
    cost = cost / m
    return cost


ตรวจสอบการใช้งานฟังก์ชันค่าใช้จ่าย (cost function) ในเซลล์ด้านล่าง

In [ ]:
w_tmp = np.array([1,1])
b_tmp = -3
print(compute_cost_logistic(X_train, y_train, w_tmp, b_tmp))

**ผลลัพธ์เป้าหมาย**: 0.3668667864055175

## ตัวอย่าง (Example)
ตอนนี้เรามาดูกันว่าค่าฟังก์ชันต้นทุน (cost function) จะเป็นอย่างไร เมื่อเราเปลี่ยนค่า $w$. 

* ในการทดลองก่อนหน้านี้ คุณได้พล็อตเส้นแบ่งข้อมูล (decision boundary) สำหรับ   $b = -3, w_0 = 1, w_1 = 1$. นั่นคือ คุณมี `b = -3, w = np.array([1,1])`.

* สมมติว่าคุณต้องการทราบว่าค่า $b = -4, w_0 = 1, w_1 = 1$, or `b = -4, w = np.array([1,1])` จะให้โมเดลที่ดีกว่า

เราจะลองพล็อตเส้นแบ่งข้อมูลสำหรับค่า  $b$ ทั้งสองนี้ เพื่อดูว่าค่าใดเหมาะสมกับข้อมูลมากกว่า

* สำหรับ $b = -3, w_0 = 1, w_1 = 1$, we'll plot $-3 + x_0+x_1 = 0$ (เเสดงสีเขียว)
* สำหรับ $b = -4, w_0 = 1, w_1 = 1$, we'll plot $-4 + x_0+x_1 = 0$ (เเสดงสีม่วง)

In [ ]:
import matplotlib.pyplot as plt

# Choose values between 0 and 6
x0 = np.arange(0,6)

# Plot the two decision boundaries
x1 = 3 - x0
x1_other = 4 - x0

fig,ax = plt.subplots(1, 1, figsize=(4,4))
# Plot the decision boundary
ax.plot(x0,x1, c=dlc["dlblue"], label="$b$=-3")
ax.plot(x0,x1_other, c=dlc["dlmagenta"], label="$b$=-4")
ax.axis([0, 4, 0, 4])

# Plot the original data
plot_data(X_train,y_train,ax)
ax.axis([0, 4, 0, 4])
ax.set_ylabel('$x_1$', fontsize=12)
ax.set_xlabel('$x_0$', fontsize=12)
plt.legend(loc="upper right")
plt.title("Decision Boundary")
plt.show()

คุณสามารถเห็นจากพล็อตนี้ว่า`b = -4, w = np.array([1,1])` เป็นโมเดลที่แย่กว่าสำหรับข้อมูลฝึกอบรม
ลองดูว่าการใช้งานฟังก์ชันต้นทุนสะท้อนถึงสิ่งนี้หรือไม่

In [ ]:
w_array1 = np.array([1,1])
b_1 = -3
w_array2 = np.array([1,1])
b_2 = -4

print("Cost for b = -3 : ", compute_cost_logistic(X_train, y_train, w_array1, b_1))
print("Cost for b = -4 : ", compute_cost_logistic(X_train, y_train, w_array2, b_2))

**ผลลัพธ์เป้าหมาย**

Cost สำหรับ b = -3 :  0.3668667864055175

Cost สำหรับ b = -4 :  0.5036808636748461


ในตัวอย่างนี้ คุณสามารถสังเกตได้ว่าค่าใช้จ่าย (cost function) มีพฤติกรรมตามที่คาดหวัง cost สำหรับ
`b = -4, w = np.array([1,1])` นั่นหมายความว่าการปรับค่า `b = -3, w = np.array([1,1])` ทำให้ cost ลดลง








## ยินดีด้วย!
ในห้องปฏิบัติการนี้ คุณได้ตรวจสอบและใช้ฟังก์ชันต้นทุนสำหรับการถดถอยโลจิสติก (logistic regression) แล้ว